---
title: PEFT
subtitle: LLM Parameter Efficient Fine Tuning with huggingface tutorial
description: LLM의 파라미터는 일일이 조정하기엔 지나치게 많다. 어떻게 fine tuning 할 수 있을까?
categories:
    - NLP
    - huggingface
    - tutorial
date: 2023.10.11
---

### Quicktour

모든 peft는 PeftConfig class를 정의한다. PeftConfig는 PeftModel을 만드는데 중요한 parameters를 저장한다. huggingface Quicktour에서는 LoRA를 이용해 클래스 분류문제를 푸므로 여기서 생성해야 할 PeftConfig는 LoraConfig다.

LoraConfig는 아래와 같이 정의한다

```py
# pip install peft
from peft import LoraConfig, TaskType
peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8,
                        lora_alpha=32, lora_dropout=-0.3 )
```

#### major parameters

각 인자는 다음을 나타낸다. ([more](https://huggingface.co/docs/peft/main/en/package_reference/tuners#peft.LoraConfig))

- `task_type` : TaskType 에서 task 종류를 정의한다. 이 경우에는 Seq2Seq 언어 모델링이다.
    - 왜 하필 Seq2Seq?
- `inference_mode` : 추론을 할 때 사용한다.
- `r` : low-rank matrices의 차원을 결정한다.
    - LoRA의 LoR가 Low Rank다. (LoRA: Low Rank Adaptation) 이 때의 r 값이 '학습 가능한 분해행렬'에 해당한다.
    - low-rank matrices: 행렬의 랭크는 행렬에서 선형적으로 독립적인 열(또는 이에 상응하는 행)의 최대 수, 행렬로 표현되는 벡터가 포함하는 최대 차원 수로 행렬에 포함된 정보의 양이라고 이해할 수 있다.
    - 아래와 같이 구할 수 있다.

        ```py
        import numpy as np
        A = np.array([[1, 2, 3], [2, 4, 6], [3, 6, 9]])
        print("Rank of A:", np.linalg.matrix_rank(A))   # 1

        B = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
        print("Rank of B:", np.linalg.matrix_rank(B))   # 2
        ```
        
- `lora_alpha` : low rank matrices를 위한 'scaling factor', lora scaling에 필요한 파라미터다.
    - lora scaling? ([LoRA 참고](https://4923.github.io/archive/NLP/LoRA.html)) : LoRA는 사전학습된 가중치 행렬 $W_0$ 이 업데이트 될 때 $\Delta W$ 를 $BA$ 로 바꾸어 더하는 과정이다.
        $$W_0 + \Delta W = W_0 + BA$$
        여기서 $A$는 random Gaussian initialized matrix 이고 B는 0으로 initialization 된 값이다. 학습 과정에서 $\Delta W x$ 는 $\frac{\alpha}{\gamma}$ 로 **scaling** 되고 이후 Optimized 된다. 여기서$\alpha$ 를 튜닝하는 것은 learning rate처럼 tuning 될 수 있다. 
- `lora_dropout` : LoRA 레이어를 dropout하는 확률이다.

다음으로 PeftModel을 정의한다. `get_peft_model()` 로 불러오며 불러오는 방식은 다른 huggingface 모델의 방식과 동일하다. 

```py
from transformers import AutoModelForSeq2SeqLM

model_name_or_path = "bigscience/mt0-large"
tokenizer_name_or_path = "bigscience/mt0-large"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
```

이렇게 기본 모델과 `peft_config`을 `get_peft_model`로 wrapping한다. 이렇게 wrap된 모델이 PeftModel이다.

```py
from peft import get_peft_model

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
"output: trainable params: 2359296 || all params: 1231940608 || trainable%: 0.19151053100118282"
```

위 코드의 실행 결과를 보면 실제로 우리가 학습시켜야 하는 학습인자는 0.19%만 되는 것을 확인할 수 있다. 원체 초기 파라미터 개수가 방대라니 0.19%도 적지 않은 수겠지만 비율상 상당한 감소다.

#### Reference

- [https://huggingface.co/blog/peft](https://huggingface.co/blog/peft)
- [https://da2so.tistory.com/79](https://da2so.tistory.com/79)